How to convert a Google sheet to a static marketing website

templates google doc?

templates google sheet?

In [ ]:
var util = require('util');
var GoogleSpreadsheet = require('google-spreadsheet');

function getTemplates(doc) {
    if(typeof doc === 'string') {
        doc = new GoogleSpreadsheet(doc);
    }
    return util.promisify(doc.getInfo)()
        .then(info => {
            console.log('Loaded doc: ' + info.title + ' by ' + info.author.email);
            return info;
        })
        .then(s => s.worksheets.reduce((obj, worksheet) => {
            var layoutTitle = worksheet.title.toLowerCase()
                .replace(/\s*layout|\s*data|\s*template/ig, '')
                .split(/s+/ig).join('-');
            if( typeof obj[layoutTitle] == 'undefined' ) obj[layoutTitle] = {template: null, data: null};
            if( worksheet.title.toLowerCase().includes('data') ) obj[layoutTitle].data = worksheet;
            else obj[layoutTitle].template = worksheet;
            return obj;
        }, {}))
}
module.exports = getTemplates;


test templates google sheet?


In [ ]:
var assert = require('assert');
var importer = require('../Core');
var getTemplates = importer.import('templates google sheet');

var docsId = '113mVIumItArQ_oXpfDRXP-2Kw2ms4t48oPJ68_p5P8k';

describe('get templates from sheet', () => {
    
    it('should have at least one page', () => {
        return getTemplates(docsId)
            .then(t => {
                console.log(JSON.stringify(Object.keys(t).reduce((obj, cur) => {
                    obj[cur] = {};
                    obj[cur].template = ((t[cur].template || {}).title || '').substr(0, 1000);
                    obj[cur].data = (t[cur].data ? ('yes: ' + t[cur].data.rowCount) : 'no');
                    return obj;
                }, {}), null, 4))
                assert(Object.keys(t).length > 0, 'should have a page called template');
            })
    })
    
    
})


get google data sheet?

google sheet array objects?


In [ ]:
var Remarkable = require('remarkable');
var md = new Remarkable({html: true, xhtmlOut: true, breaks: true});
var util = require('util');

function getDataSheet(worksheet) {
    if(!worksheet) {
        return Promise.resolve(false);
    }
    return util.promisify(worksheet.getCells)({
        'min-row': 1,
        'max-row': 100,
        'return-empty': false
    })
        .then(cells => {
            var columns = [];
            return cells.reduce((acc, o, arr) => {
                if (o.row == 1) {
                    columns[o.col] = o.value;
                    return acc;
                }
                if (!acc[o.row - 2]) {
                    acc[o.row - 2] = {};
                }
                var html = md.render(o.value);
                // if all markdown did was insert a paragraph and line break, use value instead
                if(html.replace(/<\/?p>|\s+/ig, '') == o.value) {
                    html = o.value;
                }
                acc[o.row - 2][columns[o.col]] = html;
                return acc;
            }, []);
        })
}
module.exports = getDataSheet;


test google data sheet?


In [ ]:
var assert = require('assert');
var importer = require('../Core');
var getTemplates = importer.import('templates google sheet');
var getDataSheet = importer.import('google sheet array objects');

var docsId = '113mVIumItArQ_oXpfDRXP-2Kw2ms4t48oPJ68_p5P8k';

describe('get data from data sheet', () => {
    
    it('should process at least one data sheet', () => {
        return getTemplates(docsId)
            .then(t => {
                var key = Object.keys(t).filter(k => t[k].data)[0];
                assert(typeof key != 'undefined', 'should have at least one data sheet');
                return getDataSheet(t[key].data);
            })
            .then(data => {
                console.log(data.length);
                console.log(data);
                assert(data.length > 0, 'should have data');
            })
    })
    
    
})


google sheet layout template?

Convert a google sheet page to an HTML template



In [1]:
var Remarkable = require('remarkable');
var md = new Remarkable({html: true, xhtmlOut: true, breaks: true});
var getDataSheet = importer.import('google sheet array objects');

// must do this up front so we can process all data
function getSections(rows, properties, templates) {
    var returnObj = {};
    var sections = rows.reduce((obj, row, i) => {
        var subsections = row.reduce((obj, c, j) => {
            var value = row[j + 1];
            if(c === ':section' && typeof templates[value] != 'undefined') {
                obj[value] = getDataSheet(templates[value].data)
                    .then(data => {
                        properties[value + '-data'] = data;
                        returnObj[value + '-data'] = data;
                        return getLayoutSheet(templates[value].template, properties, templates);
                    })
                    .then(template => {
                        properties[value] = template;
                        returnObj[value] = template;
                    });
            }
            return obj;
        }, obj);
        return obj;
    }, {});
    return Promise.all(Object.values(sections)).then(() => returnObj);
}

function getRows(cells) {
    // turn cells in to rows, TODO: can getCells options do this for me?
    var rows = cells.reduce((arr, c) => {
        if(typeof arr[c.row - 1] == 'undefined') arr[c.row - 1] = [];
        arr[c.row - 1][c.col - 1] = c.value;
        return arr;
    }, []);
    return rows;
}

function getDataClasses(c, data) {
    // get classes from mustache vars used with supplied data
    var dataKeys = (data || []).reduce((keys, cur) => {
        // get all keys in data
        return keys.concat(Object.keys(cur));
    }, []).filter((k, h, a) => a.indexOf(k) == h);
    var dataClasses = dataKeys
        .filter(k => ('\\{\\{' + k + '\\}\\}', 'ig').match(c))
        .map(k => 'val-' + k)
        .join(' ')
    return dataClasses;
}

function getLayoutSheet(worksheet, properties, templates) {
    if(!worksheet) {
        return Promise.resolve(false);
    }
    var layoutTitle = worksheet.title.toLowerCase()
                .replace(/\s*layout|\s*data|\s*template/ig, '')
                .split(/s+/ig).join('-');
    var rows;
    return util.promisify(worksheet.getCells)({
        'min-row': 1,
        'max-row': 100,
        'return-empty': false
    })
        .then(cells => {
            // get all sub sections up front
            rows = getRows(cells);
            return getSections(rows, properties, templates);
        })
        .then(sections => {
            // set object properties for mustache template
            var html = rows.reduce((arr, row, i) => {
                var rowsHtml = row.reduce((arr, c, j) => {
                    if(c.substr(0, 1) === ':') {
                        // use subsequent column for property values
                        var value = row[j + 1];
                        if(c.substr(1).toLowerCase() === 'section') {
                            arr[arr.length] = `
<div class="section-${value}">
${sections[value]}
</div>
`;
                        } else {
                            properties[c.substr(1)] = value;
                        }
                    // render markdown content if it is not the value for the previous property
                    } else if(j == 0 || j >= 1 && row[j - 1]
                              && row[j - 1].substr(0, 1) !== ':') {
                        var dataClasses = getDataClasses(c, properties[layoutTitle + '-data']);
                        
                        var mdHtml = md.render(c);
                        // if all markdown did was insert a paragraph and line break, use value instead
                        if(mdHtml.replace(/<\/?p>|\s+/ig, '') == c) {
                            mdHtml = c;
                        }
                        arr[arr.length] = `
<div class="${dataClasses} cell-${j}">
${mdHtml}
</div>
`;
                    }
                    return arr;
                }, []);
                
                // render mustache templates
                if(rowsHtml.length > 0) {
                    var rowTemplate = `
<div class="${properties['class'] || ''} col-${rowsHtml.length}">
${rowsHtml.join('')}
</div>
`;
                    arr[arr.length] = rowTemplate;
                }
                
                return arr;
            }, []);
        
            // render mustache page template
            return html.join('');
        })
}

module.exports = getLayoutSheet;

{ getLayoutSheet: [Function: getLayoutSheet],
  htmlSheetMarkup: [Function: htmlSheetMarkup] }

test google sheet template?


In [ ]:
var assert = require('assert');
var importer = require('../Core');
var getTemplates = importer.import('templates google sheet');
var getLayoutSheet = importer.import('google sheet layout template');
var getDataSheet = importer.import('google sheet array objects');

var docsId = '113mVIumItArQ_oXpfDRXP-2Kw2ms4t48oPJ68_p5P8k';

describe('get html from template', () => {
    
    it('should process at least one template', () => {
        var templates, key, properties = {};
        return getTemplates(docsId)
            .then(t => {
                templates = t;
                key = Object.keys(templates).filter(k => templates[k].template)[0];
                return getDataSheet(templates[key].data);
            })
            .then(data => {
                assert(typeof key != 'undefined', 'should have at least one template');
                properties[key + '-data'] = data;
                return getLayoutSheet(templates[key].template, properties, templates);
            })
            .then(template => {
                console.log(template);
                console.log(properties);
                assert(template.length > 0, 'should have a page called template');
            })
    })
    
    
})


output google sheet template?

Save the generated template to an HTML file, wrapping it in a base template


In [ ]:
var fs = require('fs');
var path = require('path');
var Mustache = require('mustache');

function writeTemplate(html, properties) {
    // TODO: automatically set title if it isn't set manually
    var result;
    if(typeof properties['title'] == 'undefined' && (result = (/<h1>(.*)<\/h1>/ig).exec(html))) {
        properties['title'] = result[1];
    }
    
    var pageHtml = `
<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1.0, user-scalable=no">
<link rel="icon" href="{{logo}}">
<link rel="stylesheet" media="screen" href="/illuminati-style.css">
<meta property="og:type" content="website">
<meta property="og:title" content="{{title}}">
<title>{{title}}</title>
<style>
html, body, input, textarea, button {
    margin: 0;
    padding: 0;
    font-family: Roboto,'Droid Sans','Google Sans Display',-apple-system,BlinkMacSystemFont,sans-serif,'Open Sans',Arial,Helvetica;
    font-size: large;
    color: #666666;
}

*, :before, :after {
    padding: 0;
    margin: 0;
    box-sizing: border-box;
}

h1, h2, h3 {
    padding: 0 20px 20px;
    text-align: center;
    font-weight: lighter;
}

iframe, img {
    width: 100%;
}

heading {
    position: fixed;
    top: 0;
    left: 0;
    right: 0;
    width: 100%;
    height: 60px;
    background: rgba(0, 0, 0, .1);
    z-index: 900;
}

heading a[href="/"] {
    width: 60px;
    display: inline-block;
}

nav {
    position: fixed;
    top: 0;
    right: 0;
    left: auto;
    width: auto;
    padding: 20px 40px;
    z-index: 901;
}

nav > a + div, nav > a + p + div {
    display: none;
    position: absolute;
    top: 60px;
    right: 10px;
    background: rgba(0, 0, 0, .8);
    border-radius: 2px;
    width: 400px;
    padding: 20px;
    color: white;
}

nav > a + div img, nav > a + p + div img {
    width: 50%;
    height: auto;
    float: left;
}

nav > a:hover + div, nav > a:hover + p + div, nav > a + div:hover, nav > a + p + div:hover {
    display: block;
}

body > div.col-1:nth-of-type(2) {
    padding: 180px 60px 120px;
    background: transparent;
    text-align: center;
}

body > div:nth-of-type(3) {
    padding: 60px;
    background: #EEEEEE;
}

body > div:nth-of-type(4) {
    padding: 60px;
    background: #FFFFFF;
}

body > .col-2, body > .col-3, body > .col-4 {
    display: flex;
    flex-direction: row;
    align-items: center;
    justify-content: center;
}

body > .col-2 > * {
    flex-basis: 50%;
    margin: 20px;
}

body > .col-3 > * {
    flex-basis: 33.33%;
    margin: 20px;
}

body > .col-4 > * {
    flex-basis: 25%;
    margin: 20px;
}

form {
    display: flex;
    flex-direction: column;
}

form > * {
    width: 100%;
}

input, textarea {
    padding: 10px;
    outline: none;
    background: rgba(0, 0, 0, 0.1);
    border-radius: 2px;
    position: relative;
    border: 2px solid transparent;
    highlight: none;
    height: 45px;
    z-index: 1;
}

label:after {
    content: " ";
    display: block;
    width: auto;
    height: 2px;
    border: 1px solid #BBBBBB;
    position: absolute;
    bottom: 0;
    top: auto;
    left: 50%;
    right: 50%;
    visibility: hidden;
    transition: ease-in-out .15s;
    transition-property: left, right, bottom;
    z-index: 2;
}

input:focus + label:after, textarea:focus + label:after {
    visibility: visible;
    left: 2px;
    right: 2px;
    bottom: -35px;
}

label {
    padding: 15px;
    font-size: medium;
    position: relative;
    top: -45px;
    transition: ease-in-out .15s;
    pointer-events: none;
    z-index: 0;
    height: 45px;
}

input:focus + label, textarea:focus + label {
    top: -80px;
    font-size: small;
}

</style>
</head>
<body class="{{class}}" style="background-image: {{banner}};">
${html}
</body>
</html>`;
    Mustache.parse(pageHtml);
    return Mustache.render(pageHtml, properties);
}

module.exports = writeTemplate;


test output sheet template?


In [ ]:
var fs = require('fs');
var path = require('path');
var assert = require('assert');
var importer = require('../Core');
var getTemplates = importer.import('templates google sheet');
var getLayoutSheet = importer.import('google sheet layout template');
var getDataSheet = importer.import('google sheet array objects');
var writeTemplate = importer.import('output google sheet template');
var PROJECT_OUTPUT = path.resolve(path.join(__dirname, "../.output"));

var docsId = '113mVIumItArQ_oXpfDRXP-2Kw2ms4t48oPJ68_p5P8k';

describe('output rendered html', () => {
    
    it('should process at least one template', () => {
        var templates, key, properties = {};
        return getTemplates(docsId)
            .then(t => {
                templates = t;
                key = Object.keys(templates).filter(k => templates[k].template)[0];
                return getDataSheet(templates[key].data);
            })
            .then(data => {
                assert(typeof key != 'undefined', 'should have at least one template');
                properties[key + '-data'] = data;
                return getLayoutSheet(templates[key].template, properties, templates);
            })
            .then(template => {
                assert(template.length > 0, 'should have a page called template');
                return writeTemplate(template, properties);
            })
            .then(page => {
                console.log(page);
                // TODO: move this in to a new function for detecting links and writing every part of site
                var filename = path.join(PROJECT_OUTPUT, key + '.html');
                fs.writeFileSync(filename, page);
                assert(page.length > 0, 'should have a page');
            })
    })
    
    
})
